In [1]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
import sys
import utils
import base64
import pandas as pd

In [2]:
client = OpenAI()

In [3]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
    Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.
    Rating 9 (Very Good Road Condition - Routine Maintenance): Very few visible defects; less than 10% surface defects (ravelling and bleeding).
    Rating 8 (Good Road Condition - Restoration of Skid Resistance): 10% to 30% surface with surface defects; no structural distresses present.
    Rating 7 (Good Road Condition - Restoration of Skid Resistance): More than 30% of surface with surface defects (ravelling or bleeding); no structural distresses.
    Rating 6 (Fair Road Condition - Surface Restoration): Less than 20% other cracking, patching in good condition, some surface distortion present.
    Rating 5 (Fair Road Condition - Surface Restoration): More than 20% surface with other cracking, patching in fair condition, significant surface distortions.
    Rating 4 (Poor Road Condition - Structural Overlay): 5% to 25% of surface with structural distresses (rutting, alligator cracking, poor patching), frequent potholes.
    Rating 3 (Poor Road Condition - Structural Overlay): 25% to 50% of surface with significant structural distresses, more frequent potholes.
    Rating 2 (Very Poor  Road Condition - Road Reconstruction): More than 50% of surface with severe structural distresses, severe rutting, many potholes.
    Rating 1 (Failed Poor  Road Condition - Road Reconstruction): Complete road failure, many large and deep potholes, extensive loss of pavement surface.
    
    Defect Categories and Indicators:
    Surface Defects: Include ravelling (loss of binder and aggregate from the surface) and bleeding (excess bituminous material on the surface).
    Pavement Deformation: Encompasses rutting (permanent deformation in wheel paths) and surface distortion (depressions, bumps).
    Cracks: Categorized into alligator cracking (series of interconnected cracks), edge breakup/cracking, and other cracking (longitudinal, transverse, reflection, slippage).
    Surface Openings: Include patching (repaired areas indicating previous pavement defects) and potholes (bowl-shaped depressions exposing underlying layers).
"""

In [9]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                    {
                        "role": "system",
                        "content":"You are a pavement engineer tasked with evaluating road conditions from images.\
                        You will also be provided with road condition grading criteria (road_grading_standards) and image.\
                        Use the following step-by-step instructions to evaluate the road grade.\
                        Step 1 – The user will provide you with road condition grading criteria.\
                        Step 2 - The user will provide you with an image in base64 format, focus solely on the asphalt pavement.\
                        Step 3 - Please pay attention to any anomalies on the asphalt pavement, such as cracks, alligator\
                        cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, patching, etc. \
                        When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting\
                        conditions.\
                        Step 4 - Using the provided road condition grading criteria determine the grade of the road surface in the image.\
                        Step 5 - Respond with a single grade number based on the provided standards. \
                        Respond with only one number."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "I will provide you with road condition grading criteria."},
                            {"type": "text", "text": road_grading_standards},
                            {"type": "text", "text": "I will provide you with an image in base64 format, please focus solely on the asphalt pavement"},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            },
                            {"type": "text", "text": "Please pay attention to any anomalies on the asphalt pavement, such as cracks, alligator\
                            cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, patching, etc.\
                            When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting\
                            conditions."},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},
                            {"type": "text", "text": "Respond with a single grade number based on the provided standards.\
                            Respond with only one number."},
                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [10]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model4_o_10131120.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)

Invalid response for 106.png: I can't evaluate the road condition from the image provided. Please provide the image in base64 format.
Invalid response for 56.png: I can't evaluate the road condition from the image provided. Please provide the image in base64 format.
